# Introduction

Explain problem, show images of trig and first, show diff

In [5]:
import numpy as np
import tensorflow as tf

# Class def. Stuff

In [3]:
class BubbleEvent:
    def __init__(self, File):
        #temp pixel arrays and event level meta data
        self.FileName = File
        Bot1PixelArray, Bot2PixelArray = BubbleEvent.GetPixelArray(self.FileName) #gets 2d array of pixel intensities
        self.Date, self.Run, self.EventID = BubbleEvent.GetRunInfo(self.FileName) #parses image name to get event info
        self.BubbleCount = 0
        #actual features to use to classify
        self.UsefulEdgeFeature0, self.UsefulEdgeFeature1, self.UsefulEdgeFeature2 = (GetEdgeFeature(
                                        DownSampleTheArray(2, Bot1PixelArray)) + 
                                        GetEdgeFeature(DownSampleTheArray(2, Bot2PixelArray))) #edge detect. sum
        self.UsefulBlobFeature = np.std(GetBlobs(Bot1PixelArray)) + np.std(GetBlobs(Bot2PixelArray)) #blob convalution deviation
        self.CountBlobPeakFeature = GetPeaks(Bot1PixelArray) + GetPeaks(Bot2PixelArray)
    def GetPixelArray(FileName):
        im = Image.open(FileName)
        PixelArray = np.asarray(im)
        Cutout = Cutout2D(PixelArray, (530,140), 235) #just cut out the parts of the image with bottles
        Bot1PixelArray = Cutout.data
        PixelArray =ndimage.rotate(PixelArray, -45)
        Cutout2 = Cutout2D(PixelArray, (270,310), 235) #other bottle view
        Bot2PixelArray = Cutout2.data
        return Bot1PixelArray, Bot2PixelArray
    def GetRunInfo(File):
        Date = int(File.split("/")[-1].split("_")[0]) #file should be date_run_event
        Run = int(File.split("/")[-1].split("_")[1])
        Event = int("{}{}{}".format(Date, Run,File.split("/")[-1].split("_")[2])) 
        return Date, Run, Event

# Perceptron Stuff

# Feature data cleaning

In our model we use the edge feature, blob feature, and blob peak feature extracted from the diff images for each event. The dataframe is first loaded from `FeaturesDataFrame.p`, which is a pickled dataframe with the information from `Events.p`. During the handscan process for the SBC, many events were assigned an "invalid" or -1 bubble count due to undesired behavior in the bubble chamber, but still have valid image information. In this case, we apply a cut which throws away around 5% of all events which have a -1 count but the peak feature is greater than 1, and the remaining -1 count events are said to have no bubbles. All events with a bubble count greater than 0 are grouped into the "have-bubble" class. Each of the features is then normalized to fit a standard gaussian distribution, and randomly split into a 80% train set and 20% test set.

| No. of bubbles | 0 | 1 | 2 | 3 | 4 | Total
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| No. of events | 16055 | 7003 | 260 | 15 | 1 | 23334 |

In [ ]:
# get x and y from dataframe
x = np.array(df[['edgefeature', 'blobfeature', 'blobpeakfeature']])
y = 1-(np.array(df['bubblecount'])<=0)
for i in range(3):
    x[:,i] = gaussian_normalize(x[:,i])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Neural net model

A simple neural net model is then trained using the training data. The input data for each event is a three number feature set extracted from the images, and the output is 0 for no bubble or 1 for having bubbles. We used the `tensorflow.keras` package to construct our model. The model has only one hidden layer of 8 nodes, and the input, hidden and output layers are densely connected. Due to the small size of input and output, a more complicated model cannot achieve higher accuracy, but takes significantly longer to run and has a higher risk of overfitting. The loss function is the categorical cross entropy, and the metric that this model is evaluated against is the total accuracy. After training this model for 20 epochs, the loss minimized to 0.1396, and accuracy on the training set arrives at 96.13%. When evaluated on the test set, it achieved a loss of 0.1324 and accuracy of 96.29%. No class-dependent weights is used as that does not seem improve accuracy. See below for the core piece of code that defines, trains and evaluates the model, and the confusion matrix on the test set of 4667 events.

In [ ]:
model = tf.keras.Sequential([ # define layers in model
    tf.keras.layers.Dense(8, activation=tf.nn.relu, input_shape=(3,)),
    tf.keras.layers.Dense(2)
])
weights = {0:1, 1:1} # define weights
model.compile(optimizer = 'adam', 
               loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
               metrics = ["accuracy"])
model.fit(x_train,y_train,epochs = 20, verbose=0) # train model on training set
test_accuracy = model.evaluate(x_test, y_test, verbose = 3) # output loss and accuracy on test set
y_pred = model.predict_classes(x_test) # get confusion matrix

| | Predicted No Bubble | Predicted Bubble |
|:---:|:---:|:---:|
| No Bubble | 3168 | 60 |
| Bubble | 115 | 1324 |

Since 3 and 4 bubble events in our dataset are rare, we could not achieve significant accuracy for those predictions. However, this model could still render useful because it can still reduce the number of events that need manual classification by a factor of 3. In the future, we could explore some other features that can better represent the number of bubbles in an image as opposed to just bubble / no bubble.